Kurulumlar

In [1]:
import os
import re
import tensorflow as tf

In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00
time: 258 µs (started: 2024-04-10 20:45:25 +00:00)


In [2]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [4]:
!git lfs install

Git LFS initialized.


In [102]:
!git config --global user.email ""
!git config --global user.name "denizzhansahin"

In [6]:
!git config --global credential.helper store

In [9]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Veri Seti İşlemleri

In [10]:
%cd /content/drive/MyDrive/Veri-Kazima/Deneme

[Errno 2] No such file or directory: '/content/drive/MyDrive/Veri-Kazima/Deneme'
/content


In [15]:
#veri setini yükle
file_name = "deneme.csv"
path = "/content/drive/MyDrive/Veri-Kazima/Deneme/"

In [16]:
#veri seti hakkında bilgi al
import pandas as pd

def kelime_sayisi(dosya_adi):
  with open(dosya_adi, "r") as dosya:
    metin = dosya.read()
  kelimeler = metin.split()
  return len(kelimeler)

dosya_kaydet = path+file_name
data = pd.read_csv(dosya_kaydet)
data = pd.DataFrame(data)
#data['Icerik'].fillna('Veri Yok', inplace=True)

In [ ]:
data.Icerik.count()

1600

In [ ]:
len(data)

1600

In [ ]:
kelime_sayisi(dosya_kaydet)

367988

In [ ]:
#data.to_csv("deneme.csv")

In [ ]:
file_name = "deneme.csv"

In [17]:
from datasets import load_dataset
reviews_dataset = load_dataset("csv", data_files= path+file_name)

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
#veri seti bilgileri
reviews_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'Baslik', 'Ozet', 'Kategori', 'Link', 'Icerik'],
        num_rows: 1600
    })
})

In [19]:
#veri setinden bir parça
reviews_dataset['train'][:5]

{'Unnamed: 0.1': [0, 1, 2, 3, 4],
 'Unnamed: 0': [0, 1, 2, 3, 4],
 'Baslik': ['BTK Akademi 11 ilde yapay zeka ve oyun geliştirme gibi alanlarda ücretsiz eğitim verecek',
  'Ticaret Bakanlığı tarafından İsrail ile uçak benzini ve jet yakıtı dahil 54 ürün grubunda ihracat kısıtlandı',
  'Cemil Kazancı hayatı ve biyografisi',
  '9 Nisan 2024 arefe günü Bankalar yarım gün çalışacak',
  "2024 ilk Güneş tutulması 8 Nisan'da Türkiye saati ile 21.20'de başladı"],
 'Ozet': ['BTK Akademi açıklanan bilgilere göre yapay zeka ve oyun geliştirme gibi alanlarda 11 ilde yüz yüze eğitim verecek. Vatandaşlar BTK akademinin hangi illerde eğitim vereceğini araştırıyor. Ayrıca eğitim verilecek olan alanların listesi de merak ediliyor. İşte, detaylar...',
  'Ticaret Bakanlığı 54 ürün grubunda İsrail\'le ihracatı kısıtladı. "İsrail\'le ticareti kısıtlanan ürünler nedir?" sorusunun cevabı vatandaşlar tarafından araştırılıyor. Ayrıca kısıtlamaların ne kadar süreceği de merak ediliyor. İşte, detaylar...',
  "Gü

In [21]:
#veri setini 4000 değerine kadar al
reviews_sample = reviews_dataset["train"].shuffle(seed=42).select(range(1559))

In [22]:
reviews_sample

Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'Baslik', 'Ozet', 'Kategori', 'Link', 'Icerik'],
    num_rows: 1559
})

In [23]:
#istenmeyen bir sütunu sil
reviews_sample = reviews_sample.remove_columns('Unnamed: 0') #sütun ismini yaz
reviews_sample = reviews_sample.remove_columns('Baslik')
reviews_sample = reviews_sample.remove_columns('Ozet')
reviews_sample = reviews_sample.remove_columns('Kategori')
reviews_sample = reviews_sample.remove_columns('Link')
reviews_sample

Dataset({
    features: ['Unnamed: 0.1', 'Icerik'],
    num_rows: 1559
})

In [24]:
reviews_sample = reviews_sample.remove_columns('Unnamed: 0.1')

In [25]:
reviews_sample

Dataset({
    features: ['Icerik'],
    num_rows: 1559
})

In [26]:
#sütun ismini değiştir
reviews_sample = reviews_sample.rename_column(
    original_column_name="Icerik", new_column_name="review" #eski sütun ismi/ yeni sütun ismi
)
reviews_sample

Dataset({
    features: ['review'],
    num_rows: 1559
})

In [27]:
#her bir satır değerini ilgili sütun için split et
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [28]:
reviews_sample = reviews_sample.map(compute_review_length)
reviews_sample[0]

Map:   0%|          | 0/1559 [00:00<?, ? examples/s]

{'review': "12 Mart günü olan depremlerin büyüklüğüne, saatine ve yerine Kandilli Rasathanesi ve AFAD son depremler listesi aracılığıyla ulaşıyoruz. Az önce deprem mi oldu, nerede gibi sorular gündemde yer alıyor. AFAD sayesinde gün içinde olan depremlerin detaylarına ulaşabiliyoruz. Detaylar...\nAZ ÖNCE DEPREM Mİ OLDU?\n12 Mart günü meydana gelen son depremler Kandilli Rasathanesi verilerine göre şöyle:\nKANDİLLİ RASATHANESİ SON DEPREMLER LİSTESİ!\nKandilli Rasathanesi son depremler listesi aracılığıyla gün içerisinde olan depremlere ulaşabilmekteyiz. Günlük ve anlık olarak verilerin paylaşılmasıyla beraber son depremler listesine erişebiliyoruz. Sarsıntı hisseden vatandaşlar ise son depremler listesini kontrol edebiliyor.\nAFAD SON DEPREMLER LİSTESİ\nAFAD aracılığıyla bugün olan depremlerin nerede, saat kaçta olduğunu öğrenebiliyoruz. 12 Mart günü meydana gelen tüm depremlere AFAD son depremler listesinden görebilmekteyiz.\nİlgili Haberler\nMalatya'da deprem oldu!",
 'review_length':

In [29]:
reviews_sample

Dataset({
    features: ['review', 'review_length'],
    num_rows: 1559
})

In [30]:
reviews_sample.sort("review_length")[:300]

{'review': ['Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yok',
  'Veri Yo

In [31]:
#her bir satır değerini ilgili sütun için filtre uygula
reviews_sample = reviews_sample.filter(lambda x: x["review_length"] > 30)
print(reviews_sample.num_rows)

Filter:   0%|          | 0/1559 [00:00<?, ? examples/s]

1487


In [32]:
reviews_sample[:13]

{'review': ["12 Mart günü olan depremlerin büyüklüğüne, saatine ve yerine Kandilli Rasathanesi ve AFAD son depremler listesi aracılığıyla ulaşıyoruz. Az önce deprem mi oldu, nerede gibi sorular gündemde yer alıyor. AFAD sayesinde gün içinde olan depremlerin detaylarına ulaşabiliyoruz. Detaylar...\nAZ ÖNCE DEPREM Mİ OLDU?\n12 Mart günü meydana gelen son depremler Kandilli Rasathanesi verilerine göre şöyle:\nKANDİLLİ RASATHANESİ SON DEPREMLER LİSTESİ!\nKandilli Rasathanesi son depremler listesi aracılığıyla gün içerisinde olan depremlere ulaşabilmekteyiz. Günlük ve anlık olarak verilerin paylaşılmasıyla beraber son depremler listesine erişebiliyoruz. Sarsıntı hisseden vatandaşlar ise son depremler listesini kontrol edebiliyor.\nAFAD SON DEPREMLER LİSTESİ\nAFAD aracılığıyla bugün olan depremlerin nerede, saat kaçta olduğunu öğrenebiliyoruz. 12 Mart günü meydana gelen tüm depremlere AFAD son depremler listesinden görebilmekteyiz.\nİlgili Haberler\nMalatya'da deprem oldu!",
  'Kızıl Goncala

In [33]:
#veriler için düzenleme yap, gereksiz olanları değiştir veya kaldır
def remove_repeated(example):
    example["review"] = example["review"].replace('...', '')
    example["review"] = example["review"].replace(',"', '. ')
    example["review"] = example["review"].replace('!.', '.')
    example["review"] = example["review"].replace('!,', '. ')
    example["review"] = example["review"].replace('"', '')
    example["review"] = re.sub('([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ]),([a-zA-Z0-9zığüşöçZİĞÜŞÖÇ])', '\\1. \\2', example["review"])

    return {"review": example["review"].replace('Devamını oku', '')}

In [34]:
reviews_sample = reviews_sample.map(remove_repeated)
reviews_sample[:3]

Map:   0%|          | 0/1487 [00:00<?, ? examples/s]

{'review': ["12 Mart günü olan depremlerin büyüklüğüne, saatine ve yerine Kandilli Rasathanesi ve AFAD son depremler listesi aracılığıyla ulaşıyoruz. Az önce deprem mi oldu, nerede gibi sorular gündemde yer alıyor. AFAD sayesinde gün içinde olan depremlerin detaylarına ulaşabiliyoruz. Detaylar\nAZ ÖNCE DEPREM Mİ OLDU?\n12 Mart günü meydana gelen son depremler Kandilli Rasathanesi verilerine göre şöyle:\nKANDİLLİ RASATHANESİ SON DEPREMLER LİSTESİ!\nKandilli Rasathanesi son depremler listesi aracılığıyla gün içerisinde olan depremlere ulaşabilmekteyiz. Günlük ve anlık olarak verilerin paylaşılmasıyla beraber son depremler listesine erişebiliyoruz. Sarsıntı hisseden vatandaşlar ise son depremler listesini kontrol edebiliyor.\nAFAD SON DEPREMLER LİSTESİ\nAFAD aracılığıyla bugün olan depremlerin nerede, saat kaçta olduğunu öğrenebiliyoruz. 12 Mart günü meydana gelen tüm depremlere AFAD son depremler listesinden görebilmekteyiz.\nİlgili Haberler\nMalatya'da deprem oldu!",
  "Kızıl Goncalar C

In [35]:
#veri setini bölümle
reviews_sample = reviews_sample.train_test_split(train_size=0.9, seed=42)
# test sütununu validation sütunu olarak ayarla
reviews_sample["validation"] = reviews_sample.pop("test")

reviews_sample

DatasetDict({
    train: Dataset({
        features: ['review', 'review_length'],
        num_rows: 1338
    })
    validation: Dataset({
        features: ['review', 'review_length'],
        num_rows: 149
    })
})

In [36]:
for key in reviews_sample["train"][0]:
    print(f"{key.upper()}: {reviews_sample['train'][0][key]}")

REVIEW: Malatya, Adıyaman ve çevre illerde hissedilen sarsıntının ardından depremin nerede ve kaç şiddetinde olduğu gündeme geldi. Bugün Malatya'da 4.7 büyüklüğünde deprem meydana geldi. İşte detaylar
MALATYA'DA DEPREM Mİ OLDU?
Malatya'nın Doğanşehir ilçesinin Küçüklü mahallesinde deprem meydana geldi. Depremin sarsıntısı çevre illerden de hissedildi. Kandilli Rasathanesi tarafından açıklanan bilgilere göre deprem 10.48 sularında meydana geldi. Depremin büyüklüğü ise 4.7 olarak açıklandı. 
REVIEW_LENGTH: 60


Veri Setini Yükleme/İndirme - Huggy Face

In [37]:
!pwd

/content


In [38]:
reviews_sample.save_to_disk('deneme123')

Saving the dataset (0/1 shards):   0%|          | 0/1338 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/149 [00:00<?, ? examples/s]

In [39]:
#veri setini yükle
#reviews_sample.push_to_hub("kullanıcı adı/alan adı")
reviews_sample.push_to_hub("denizzhansahin/deneme")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/denizzhansahin/deneme/commit/fd7adc26541b624cd78cb9872b73b233d20e298a', commit_message='Upload dataset', commit_description='', oid='fd7adc26541b624cd78cb9872b73b233d20e298a', pr_url=None, pr_revision=None, pr_num=None)

In [40]:
#veri setini indir
#downloaded_dataset = load_dataset("kullanıcı adı/alan adı")
downloaded_dataset = load_dataset("denizzhansahin/deneme")
downloaded_dataset

Generating train split:   0%|          | 0/1338 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/149 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'review_length'],
        num_rows: 1338
    })
    validation: Dataset({
        features: ['review', 'review_length'],
        num_rows: 149
    })
})

**Tokenizer Eğitimi**
Token : kelime sayısı

In [41]:
#hazır tokenizer kullanmak
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained("redrussianarmy/gpt2-turkish-cased") #model içindeki tokenizer al

outputs = pretrained_tokenizer(
    reviews_sample["train"][:2]["review"],#train içindeki verinin ilk 2'sini al ve review'e göre işlem yap
    truncation=True, #uzun kelimeleri alma
    max_length=context_length, #max uzunluk için
    return_overflowing_tokens=False, #40 tokendan sonra geri kalan ile işlemler
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/594k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Input IDs length: 2
Input chunk lengths: [40, 40]


In [42]:
print("vocab_size: ", len(pretrained_tokenizer)) #sözlük büyüklüğü, indirilen

vocab_size:  50258


In [43]:
txt = "Sürat Kargom Hala Gelmedi,1402 numaralı kargom adatepe şubesinde."
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)
#tokenların indexleri, indirilen veri setinden

[11283, 304, 1069, 75, 512, 20172, 2225, 4658, 16, 3168, 5299, 6358, 12546, 512, 989, 3489, 4638, 729, 18]


In [44]:
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)
#indexler ile ilgili veriler

['SÃ¼r', 'at', 'ĠKar', 'g', 'om', 'ĠHala', 'ĠGel', 'medi', ',', '14', '02', 'ĠnumaralÄ±', 'Ġkarg', 'om', 'Ġada', 'tepe', 'ĠÅŁub', 'esinde', '.']


Kendi Tokenizer'ını Eğit, Kendi Veri Setin İle

In [46]:
def get_training_corpus():
    batch_size = 1000
    return (
        reviews_sample["train"][i : i + batch_size]["review"]
        for i in range(0, len(downloaded_dataset["train"]), batch_size) #train'den parça parça veriyi al
    )
training_corpus = get_training_corpus()

In [47]:
training_corpus

<generator object get_training_corpus.<locals>.<genexpr> at 0x7e7b497e8740>

In [48]:
for reviews in get_training_corpus():
    print(len(reviews)) #paketlere bak

1000
338


In [ ]:
#kendi tokenizer'ini eğit

In [49]:
#hazır bir model ayarından , belirlenen token sayısına göre tokenizer eğitimi
vocab_size = 52000 #boyut
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size) #ayarları yap, indirilen modelden

In [50]:
tokenizer.eos_token_id #0 verisi alabilirsin

0

In [51]:
tokenizer.vocab_size #token sayısı

52000

In [52]:
txt = "Sürat Kargom Hala Gelmedi,1402 numaralı kargom adatepe şubesinde."
tokens = tokenizer(txt)['input_ids']
print(tokens) #kendi veri setinde göre indexler

[2310, 287, 953, 71, 792, 18843, 1002, 2312, 12, 2123, 5387, 8559, 6549, 792, 593, 3150, 3030, 703, 14]


In [53]:
converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['SÃ¼r', 'at', 'ĠKar', 'g', 'om', 'ĠHala', 'ĠGel', 'medi', ',', '14', '02', 'ĠnumaralÄ±', 'Ġkarg', 'om', 'Ġada', 'tepe', 'ĠÅŁub', 'esinde', '.']


In [54]:
print(len(tokenizer.tokenize(txt)))
print(len(pretrained_tokenizer.tokenize(txt)))

19
19


In [55]:
#token kaydet
path="./"
file_name="deneme_tokenizer"
tokenizer.save_pretrained(path+file_name)

('./deneme_tokenizer/tokenizer_config.json',
 './deneme_tokenizer/special_tokens_map.json',
 './deneme_tokenizer/vocab.json',
 './deneme_tokenizer/merges.txt',
 './deneme_tokenizer/added_tokens.json',
 './deneme_tokenizer/tokenizer.json')

In [56]:
#token yükle
loaded_tokenizer = AutoTokenizer.from_pretrained("./deneme_tokenizer")

In [57]:
txt = "Malatya, Adıyaman ve çevre illerde hissedilen sarsıntının ardından depremin nerede ve kaç şiddetinde olduğu gündeme geldi."
tokens = tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)


trained tokenizer: [8600, 12, 3713, 305, 7271, 5831, 9600, 9660, 750, 3190, 1823, 305, 837, 6684, 942, 3031, 919, 14]
loaded tokenizer: [8600, 12, 3713, 305, 7271, 5831, 9600, 9660, 750, 3190, 1823, 305, 837, 6684, 942, 3031, 919, 14]


In [ ]:
#token gönder, huggy face
from huggingface_hub import notebook_login
notebook_login()

In [58]:
#token gönder, huggy face
os.environ["TOKENIZERS_PARALLELISM"] = "false"
tokenizer.push_to_hub("denizzhansahin/deneme")

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/denizzhansahin/deneme/commit/7c1c93e4e632e4f6083a0bd71c754c0694f6cc9f', commit_message='Upload tokenizer', commit_description='', oid='7c1c93e4e632e4f6083a0bd71c754c0694f6cc9f', pr_url=None, pr_revision=None, pr_num=None)

In [59]:
#token indir
downloaded_tokenizer = AutoTokenizer.from_pretrained("denizzhansahin/deneme")

tokenizer_config.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/977k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/623k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [60]:
txt = "Malatya, Adıyaman ve çevre illerde hissedilen sarsıntının ardından depremin nerede ve kaç şiddetinde olduğu gündeme geldi.."
tokens = tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)
tokens = downloaded_tokenizer(txt)['input_ids']
print("downloaded tokenizer:", tokens)

trained tokenizer: [8600, 12, 3713, 305, 7271, 5831, 9600, 9660, 750, 3190, 1823, 305, 837, 6684, 942, 3031, 919, 7704]
loaded tokenizer: [8600, 12, 3713, 305, 7271, 5831, 9600, 9660, 750, 3190, 1823, 305, 837, 6684, 942, 3031, 919, 7704]
downloaded tokenizer: [8600, 12, 3713, 305, 7271, 5831, 9600, 9660, 750, 3190, 1823, 305, 837, 6684, 942, 3031, 919, 7704]


**Hugging Face Veri Hattı, Data Pipeline**
tensorFlow, keras formatına çevirme

In [61]:
#gelen text'in tokenlarını al, ve gönder
def tokenize(element):
    outputs = tokenizer(
        element["review"],
        truncation=True,
        max_length=context_length, #40'dan fazla token'ı kes
        return_overflowing_tokens=False, #arta kalan tokenlardan yeni bilgi yaratma
        return_length=True, #uzunluğu gönder
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]): #zipleme yap, uzunluk ve index
        if length == context_length: #uzunluk ve 40 token eşit ise, inputa ekle
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = downloaded_dataset.map(
    tokenize, batched=True, remove_columns=downloaded_dataset["train"].column_names #çıktıdaki kolonları düzenle
)
tokenized_datasets

Map:   0%|          | 0/1338 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1338
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 149
    })
})

Data Collator, Veriyi batch'ler halinde hazırla

In [62]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf") #tokenizer'ı al model bir şey false, tensor geri döndürme ise tf, metin üretimi için

In [63]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: (5, 40)
attention_mask shape: (5, 40)
labels shape: (5, 40)


In [64]:
for key in out:
    print(f"{key}: {out[key][0]}")

input_ids: [ 8600    12  3713   305  7271  5831  9600  9660   750  3190  1823   305
   837  6684   942  3031   919    14  2225  5061   402    65   604    14
    23  2598   672  1178   919    14   920  1577   199 15064     7   656
  2205  1564  1500    31]
attention_mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
labels: [ 8600    12  3713   305  7271  5831  9600  9660   750  3190  1823   305
   837  6684   942  3031   919    14  2225  5061   402    65   604    14
    23  2598   672  1178   919    14   920  1577   199 15064     7   656
  2205  1564  1500    31]


In [65]:
#tensorflow objesine çevir
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)
tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

In [66]:
len(tf_train_dataset) #batch sayısı

42

**Model Eğitimi**

In [67]:
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig

#gpt-2 modelini indir, mimarisini indir, configurasyonu indir, ağırlık indirmeden, sıfırdan eğit
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),#boyutunu ise tokenizer kadar yap
    n_ctx=context_length, #40 olması için
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    #başlangiç ve bitiş tokenizerları ile
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [68]:
#model indir
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)  # Builds the model
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  125778432 
 er)                                                             
                                                                 
Total params: 125778432 (479.81 MB)
Trainable params: 125778432 (479.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#huggy face'e yükle
from huggingface_hub import notebook_login
notebook_login()

In [69]:
#model eğitimi için
from transformers import create_optimizer
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,#başlaması için
    num_warmup_steps=1_000,#başlangıç için
    num_train_steps=num_train_steps,#her adımda
    weight_decay_rate=0.01, #ağırlık azalt
)

In [70]:
model.compile(optimizer=optimizer) #kerasın özelliği ile compile et, optimizer ile

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16") #gpu ile eğit

In [71]:
from transformers.keras_callbacks import PushToHubCallback
#model eğitimi sırasında, istersen ama pek tavsiye etmem
#callback = PushToHubCallback(output_dir="denizzhansahin/deneme", tokenizer=tokenizer) #huggy face'e yükle, callback yaz, her epoch'dan sonrra, validation bittikten sonra

In [88]:
#model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=3, callbacks=[callback]) #model eğitimi
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=30) #model eğitimi

Epoch 1/30
42/42 [==============================] - 20s 468ms/step - loss: 9.6934 - val_loss: 9.6704
Epoch 2/30
42/42 [==============================] - 21s 489ms/step - loss: 9.4105 - val_loss: 9.4160
Epoch 3/30
42/42 [==============================] - 21s 491ms/step - loss: 9.1023 - val_loss: 9.1460
Epoch 4/30
42/42 [==============================] - 21s 499ms/step - loss: 8.7843 - val_loss: 8.8812
Epoch 5/30
42/42 [==============================] - 20s 475ms/step - loss: 8.4798 - val_loss: 8.6399
Epoch 6/30
42/42 [==============================] - 21s 505ms/step - loss: 8.2009 - val_loss: 8.4204
Epoch 7/30
42/42 [==============================] - 21s 501ms/step - loss: 7.9445 - val_loss: 8.2215
Epoch 8/30
42/42 [==============================] - 22s 518ms/step - loss: 7.7117 - val_loss: 8.0412
Epoch 9/30
42/42 [==============================] - 21s 492ms/step - loss: 7.4965 - val_loss: 7.8807
Epoch 10/30
42/42 [==============================] - 20s 488ms/step - loss: 7.2929 - val_lo

In [89]:
#istersen bunun yerine model eğitimi bittikten sonra
model.push_to_hub("denizzhansahin/deneme")

tf_model.h5:   0%|          | 0.00/503M [00:00<?, ?B/s]

**Modelden Metin Üretimi**

In [90]:
from transformers import pipeline
from transformers import AutoTokenizer, TFGPT2LMHeadModel, AutoConfig
from datasets import load_dataset

dataset = load_dataset("denizzhansahin/deneme", split="validation")
review_model = TFGPT2LMHeadModel.from_pretrained("denizzhansahin/deneme")
review_tokenizer = AutoTokenizer.from_pretrained("denizzhansahin/deneme")

pipe = pipeline(
    "text-generation", model=review_model, tokenizer=review_tokenizer, device=0
)

tf_model.h5:   0%|          | 0.00/503M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at denizzhansahin/deneme.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [91]:
dataset

Dataset({
    features: ['review', 'review_length'],
    num_rows: 149
})

In [92]:
dataset['review'][:10]

['Bugün hangi maçların oynanacağı araştırılırken maçlar saat kaçta merak edildi. Heyecanla takip edilen liglerde günün maç fikstürü gündemde yer aldı. Taraftarlar ise bugün maç var mı sorusuna cevap arıyor.\nBUGÜN MAÇ VAR MI?\n1 Nisan akşamı oynanacak olan mücadeleler:\nİtalya Serie A \nBologna – Salernitana | 13.30 | S Sport Plus, CBC Sport, S Sport 2\nCagliari – Verona | 16.00 | S Sport Plus\nSassuolo – Udinese | 16.00 | S Sport Plus, CBC Sport, S Sport 2\nLecce – Roma | 19.00 | S Sport Plus, CBC Sport, S Sport 2\nInter – Empoli | 21.45 | S Sport Plus, CBC Sport, S Sport 2\nİspanya La Liga\nVillarreal – Atletico Madrid | 22.00 | S Sport Plus, S Sport\nLeeds United – Hull City | 22.00 | Yayıncısı yok\nSuudi Arabistan Pro Lig\nAl Ahli – Al Ittihad | 22.00 | S Sport Plus, TV 8 Buçuk, Exxen',
 "Şanlıurfa'da dünyaya gelen Kasım Gülpınar'ın kim olduğu ve nereli olduğu araştırılanlar arasında yer aldı. Son günlerde ismine sıklıkla rastladığımız Kasım Gülpınar'ın hayatı ve biyografisi araştı

In [93]:
prompts = ["Bugün hangi maçların oynanacağı araştırılırken maçlar saat kaçta merak edildi.","Şanlıurfa'da dünyaya gelen Kasım Gülpınar'ın kim olduğu ve nereli olduğu araştırılanlar arasında yer aldı."]

In [94]:
output0=pipe(prompts, num_return_sequences=1)[0][0]["generated_text"]
output1=pipe(prompts, num_return_sequences=1)[1][0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [95]:
print("For prompt ", prompts[0], " the generated text is:")
print(output0)
print("For prompt ", prompts[1], " the generated text is:")
print(output1)

For prompt  Bugün hangi maçların oynanacağı araştırılırken maçlar saat kaçta merak edildi.  the generated text is:
Bugün hangi maçların oynanacağı araştırılırken maçlar saat kaçta merak edildi. İşte'da yer aldı. 2024 Süper başlayan sarsıntı isim bugün hangi hangi gün yayınlanan UEFA ne zaman ve hangi kaç yaşında?
RAMAZAN'da oynanacak var mı?
a son?
 UEFA gelen
For prompt  Şanlıurfa'da dünyaya gelen Kasım Gülpınar'ın kim olduğu ve nereli olduğu araştırılanlar arasında yer aldı.  the generated text is:
Şanlıurfa'da dünyaya gelen Kasım Gülpınar'ın kim olduğu ve nereli olduğu araştırılanlar arasında yer aldı. İşte, ne zaman Bugün’nın her Lig'da kaç 1 Nisan'da da araştırıldı. İşte- araştırılıyor. İşte detaylar
BUGÜN
 Erdoğan yılında Avrupa


In [96]:
prompts = ["Futbol maçı hakkında bilgi ver","Bilgi nedir?"]

In [97]:
output0=pipe(prompts, num_return_sequences=1)[0][0]["generated_text"]
output1=pipe(prompts, num_return_sequences=1)[1][0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [98]:
print("For prompt ", prompts[0], " the generated text is:")
print(output0)
print("For prompt ", prompts[1], " the generated text is:")
print(output1)

For prompt  Futbol maçı hakkında bilgi ver  the generated text is:
Futbol maçı hakkında bilgi ver devam ederken İstanbul'da maç araştırmaya başladı. Birçok 2024 nerede olduğu da yer aldı. Futbolseverlerin yerel kaç yerel seçim hangi kanalda soruları yer alıyor. İşte detaylar
KIZILCIK

 Kandilli
5 İstanbul'da dizi ve AFAD'de
For prompt  Bilgi nedir?  the generated text is:
Bilgi nedir? deprem mi, nerede ve AFAD, vatandaşlar
 Lig'de oldu. detaylar
 Türkiye'de yayınlanan, son DEPREM’ın en çok mi oldu. Deprem 7 Mart günü olan 
AK
 10 Mart günü çeyrek çeyrek altın, nerede


In [99]:
prompts = ["Futbol maç","Hatfa"]

In [100]:
output0=pipe(prompts, num_return_sequences=1)[0][0]["generated_text"]
output1=pipe(prompts, num_return_sequences=1)[1][0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [101]:
print("For prompt ", prompts[0], " the generated text is:")
print(output0)
print("For prompt ", prompts[1], " the generated text is:")
print(output1)

For prompt  Futbol maç  the generated text is:
Futbol maç devam ederken bu akşam olan hangi neden araştırıldı. 2024 oynanacak olan Ligi'in olduğu merak ediliyor. Son olarak maç ile birlikte, nerede da merak edilenler arasında ise kaç yaşında, kaç devam Erdoğan'ın kaç yaşında kim da kaç yaşında oldu mı?
For prompt  Hatfa  the generated text is:
Hatfa, yayınlanan yer alan karakterini dizisinin ilk bugün ilgili hangi olarak hangi Çapkını nedeniyle merak ediliyor. 
1 Türkiye'in yeni'da neden alan dizinin ve nereli? 
GÖNÜL OLDU?
GELİNİM Ramazan son gün yer alan dizi. İşte,


In [ ]:
prompts = [dataset['review'][1],dataset['review'][1]]

In [ ]:
output0=pipe(prompts, num_return_sequences=1)[0][0]["generated_text"]
output1=pipe(prompts, num_return_sequences=1)[1][0]["generated_text"]

In [ ]:
print("For prompt ", prompts[0], " the generated text is:")
print(output0)
print("For prompt ", prompts[1], " the generated text is:")
print(output1)